# Import dependencies and libraries

In [ ]:
import os
import json
import pickle as pickle
import numpy as np

# for fitting empirical distributions for radius of gyration and jump length
import powerlaw

# for visualization
import matplotlib
import matplotlib.pyplot as plt

# trackintel -> the functions will be imported with full names
import trackintel as ti


In [ ]:
# options for better printing and visualizing

colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

matplotlib.rcParams["figure.dpi"] = 300
matplotlib.rcParams["xtick.labelsize"] = 13
matplotlib.rcParams["ytick.labelsize"] = 13
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

%matplotlib inline

In [ ]:
# get the data dir from the config file -> default stored in Data/Geolife/Data folder

DATA_DIR = os.path.join("..", "paths.json")
with open(DATA_DIR) as json_file:
    CONFIG = json.load(json_file)

save_dir = os.path.join("..", CONFIG["data_dir"])

# Load Geolife data
Trackintel provides a function [read_geolife](https://trackintel.readthedocs.io/en/latest/modules/io.html#trackintel.io.read_geolife) to directly load [Geolife](https://www.microsoft.com/en-us/research/publication/geolife-gps-trajectory-dataset-user-guide/) dataset into the trackintel positionfixes. For the tutorial and interactive section, we provided you a sample dataset with 20 selected users from the geolife dataset. This can be directly loaded from the `save_dir`. 

In [ ]:

pfs, mode_labels = ti.io.read_geolife(save_dir, print_progress=True)

## Reading your own data
Trackintel provides an I/O module for accessing movement data and storing intermediate or final results in a file or database. Three methods for converting movement data with attached attribute information to Trackintel-compatible formats are provided:
1) reading from Pandas Dataframes and Geopandas Geodataframes (**recommended**)
2) reading and writing from CSV file formats
3) reading and storing from PostgreSQL databases with PostGIS extension. Check the [input/output](https://trackintel.readthedocs.io/en/latest/modules/io.html) module for more information. 

An important consideration before loading data is to find the appropriate movement level for your data. Depending on the semantics of the data, you should call different reading functions, e.g., [read_positionfixes_csv](https://trackintel.readthedocs.io/en/latest/modules/io.html#trackintel.io.read_positionfixes_csv) or [read_staypoints_csv](https://trackintel.readthedocs.io/en/latest/modules/io.html#trackintel.io.read_staypoints_csv). 



In [ ]:
# TODO: use ti.read_positionfixes_csv() or ti.read_staypoints_csv() for loading your data. Check the documentation for the required input parameters
# selected_pfs = ti.read_positionfixes_csv()

# Calculate Radius of Gyration

## Generate staypoints
Radius of gyration calculates the characteristic distance traveled by individuals from a set of stop points, which corresponds to the staypoint data model in trackintel. The first step is thus to generate the staypoints from the original positionfixes with the trackintel [generate_staypoints](https://trackintel.readthedocs.io/en/latest/modules/preprocessing.html#trackintel.preprocessing.generate_staypoints) function.

In [ ]:
pfs, sp = pfs.generate_staypoints(gap_threshold=24 * 60, include_last=True, print_progress=True, dist_threshold=200, time_threshold=30, n_jobs=-1)

## Calculate radius of gyration with trackintel functions
Trackintel provides the function [ti.analysis.radius_gyration](https://trackintel.readthedocs.io/en/latest/modules/analysis.html#trackintel.analysis.radius_gyration) to calculate user level radius of gyration from staypoints. The function supports weighting the staypoints with their stayed duration during calculation, such that important locations (i.e., locations stayed longer; home or work) receive higher weights.

In [ ]:
rg_count = ti.analysis.radius_gyration(sp, method="count")

In [ ]:
sp["duration"] = (sp["finished_at"] - sp["started_at"])
rg_duration = ti.analysis.radius_gyration(sp, method="duration")

In [ ]:
# check the results
rg_duration

## Visualize with powerlaw package
The [powerlaw](https://github.com/jeffalstott/powerlaw) package provides functionalities to fit power-law, truncated power law, and log normal distributions to the original data. It also provides functionalities to plot all the fitted distributions on the same plot. We use the powerlaw package to visualize the radius of calculation result. 

In [ ]:
plt.figure(figsize=(8, 5))


# fit power law
xmin = 100
fit = powerlaw.Fit(rg_duration, xmin=xmin)

# plotting
powerlaw.plot_pdf(rg_count, label="count")
powerlaw.plot_pdf(rg_duration, label="duration")
fit.power_law.plot_pdf(linestyle="--", label="powerlaw fit of duration")
fit.truncated_power_law.plot_pdf(linestyle="--", label="truncated power law of duration")
fit.lognormal.plot_pdf(linestyle="--", label="lognormal fit of duration")

xlabel = "$Rg$ (m)"
ylabel = "$P(Rg)$"

plt.legend(prop={"size": 13})
plt.xlabel(xlabel, fontsize=16)
plt.ylabel(ylabel, fontsize=16)

plt.show()
